In [ ]:
pip install rasterio cfgrib pyhdf netCDF4 gdal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.3/780.3 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.4 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import zipfile
import h5py
import re
from datetime import datetime
import calendar
from glob import glob
import tarfile

import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from scipy.interpolate import RegularGridInterpolator, LinearNDInterpolator
from scipy.spatial.distance import cdist

import rasterio
from netCDF4 import Dataset, num2date
from pyhdf.SD import SD, SDC
from osgeo import gdal
import cfgrib


import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

## OBS

In [ ]:
df = pd.read_excel('/content/drive/MyDrive /01-CURAH_HUJAN/Bulanan/PHU_Bul-Pantura_1991-2024_FIX.xlsx', sheet_name = 'Sheet1')

df['Tahun'] = df['Tahun'].ffill()

# Ubah kolom Tahun dan Bulan menjadi integer
df['Tahun'] = df['Tahun'].astype(int)
df['Bulan'] = df['Bulan'].astype(int)

# Gabungkan menjadi datetime
df['date'] = pd.to_datetime(df['Tahun'].astype(str) + '-' + df['Bulan'].astype(str) + '-01')
df = df.drop(columns=['Tahun', 'Bulan'])
df

,SUBAH,BLADO SP,TERSONO,LARANGAN,NAMBO BD,KARANGSEMBUNG,CIMUNDING,LOSARI,BANTARKAWUNG,BANYUMENENG,...,WARUREJO,PESAYANGAN,METEO TEGAL,SEMARANG KLIM,SUMURJURANG,STAMET CILACAP,STAGEOF JOGJA,STAGEOF BANJARNEGARA,STAMAR TJ. EMAS,date
0,760.0,807.0,715,842.0,983.0,681.0,375.151515,345.0,320.0,343.181818,...,485.0,638.0,591.0,247.0,840.0,531.0,NaN,NaN,245.6,1991-01-01
1,1208.0,1435.0,1095,278.0,413.0,217.0,312.393939,157.0,329.0,293.787879,...,461.0,420.0,171.0,662.0,417.0,169.0,NaN,NaN,550.6,1991-02-01
2,135.0,234.0,87,188.0,136.0,273.0,336.242424,115.0,278.0,278.242424,...,569.0,254.0,201.0,111.0,176.0,275.0,NaN,NaN,133.3,1991-03-01
3,199.0,365.0,257,240.0,375.0,168.0,195.363636,139.0,182.0,250.333333,...,680.0,179.0,156.0,241.0,353.0,709.0,NaN,NaN,157.6,1991-04-01
4,91.0,139.0,72,93.0,67.0,62.0,103.242424,28.0,56.0,134.363636,...,286.0,19.0,17.0,87.0,0.0,104.0,NaN,NaN,58.1,1991-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,5.0,92.0,0,44.0,69.0,1.0,25.000000,0.0,0.0,117.000000,...,22.0,1.0,1.0,49.0,66.0,29.0,0.0,9.2,85.2,2024-08-01
404,171.0,242.0,64,125.0,70.0,133.0,25.000000,10.0,108.0,250.000000,...,60.0,53.0,25.0,59.0,108.0,95.0,13.5,179.0,77.6,2024-09-01
405,18.0,188.5,27,97.0,38.0,19.0,8.000000,0.0,28.0,185.000000,...,0.0,2.0,4.2,191.0,120.0,60.0,17.3,124.8,157.2,2024-10-01
406,247.0,578.0,224,383.0,413.0,405.0,231.000000,103.0,264.0,636.000000,...,133.0,131.0,79.0,257.0,257.0,566.0,738.9,670.4,340.7,2024-11-01


In [ ]:
# Ambil semua kolom kecuali 'date'
value_vars = df.columns[df.columns != 'date']

# Convert ke list jika perlu
value_vars = value_vars.tolist()

df = pd.melt(df,
             id_vars=['date'],
             value_vars=value_vars,
             var_name='pos',
             value_name='CH')
df = df.dropna()


,date,pos,CH
0,1991-01-01,SUBAH,760.0
1,1991-02-01,SUBAH,1208.0
2,1991-03-01,SUBAH,135.0
3,1991-04-01,SUBAH,199.0
4,1991-05-01,SUBAH,91.0
...,...,...,...
22435,2024-08-01,STAMAR TJ. EMAS,85.2
22436,2024-09-01,STAMAR TJ. EMAS,77.6
22437,2024-10-01,STAMAR TJ. EMAS,157.2
22438,2024-11-01,STAMAR TJ. EMAS,340.7


In [ ]:
df = df.dropna()
df.to_csv('/content/drive/MyDrive /01-CURAH_HUJAN/Bulanan/PHU_Long.csv', index= False)
df

,date,pos,CH
0,1991-01-01,SUBAH,760.0
1,1991-02-01,SUBAH,1208.0
2,1991-03-01,SUBAH,135.0
3,1991-04-01,SUBAH,199.0
4,1991-05-01,SUBAH,91.0
...,...,...,...
22435,2024-08-01,STAMAR TJ. EMAS,85.2
22436,2024-09-01,STAMAR TJ. EMAS,77.6
22437,2024-10-01,STAMAR TJ. EMAS,157.2
22438,2024-11-01,STAMAR TJ. EMAS,340.7


## CHIRPS

In [ ]:
# 🔹 Konfigurasi folder
source_folder = '/content/drive/MyDrive /01-CURAH_HUJAN/02-CHIRPS/Unduhan'
extract_folder = '/content/ekstrak'
output_folder = '/content/final_output'
os.makedirs(extract_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# 🔹 Batas koordinat wilayah Indonesia
lat_min, lat_max = -12.0, -5.5
lon_min, lon_max = 105.0, 115.0

tar_files = [f for f in os.listdir(source_folder) if f.endswith(".tar.gz")]

def extract_date_flexible(filename):
    # Modifikasi sesuai pola nama file kamu
    import re
    match = re.search(r"\d{4}[\._-]?\d{2}", filename)
    return match.group(0).replace("_", "-").replace(".", "-") if match else "unknown"

for tar_file in tar_files:
    try:
        tar_path = os.path.join(source_folder, tar_file)
        with tarfile.open(tar_path, "r:gz") as tar:
            members = tar.getmembers()
            for m in members:

            # Ekstrak semua file .bil dan file pendukung lainnya
            relevant_exts = [".bil", ".hdr", ".blw", ".prj"]
            extracted_names = []

            for member in members:
                if any(member.name.endswith(ext) for ext in relevant_exts):
                    member.name = os.path.basename(member.name)  # Hilangkan path folder
                    tar.extract(member, path=extract_folder)
                    extracted_names.append(member.name)


            # Proses semua file .bil yang telah diekstrak
            for filename in extracted_names:
                if not filename.endswith(".bil"):
                    continue

                bil_file_path = os.path.join(extract_folder, filename)

                with rasterio.open(bil_file_path) as dataset:
                    data = dataset.read(1)
                    data[data == dataset.nodata] = -10

                    transform = dataset.transform
                    nrows, ncols = dataset.height, dataset.width

                    lons = np.array([transform[2] + i * transform[0] for i in range(ncols)])
                    lats = np.array([transform[5] + j * transform[4] for j in range(nrows)])

                    lon_grid, lat_grid = np.meshgrid(lons, lats)

                    df = pd.DataFrame({
                        "lon": lon_grid.flatten(),
                        "lat": lat_grid.flatten(),
                        "ch": data.flatten()
                    })

                    df = df[
                        (df["lat"].between(lat_min, lat_max)) &
                        (df["lon"].between(lon_min, lon_max))
                    ].copy()

                    df["source_file"] = filename
                    df["date"] = extract_date_flexible(filename)

                    # Simpan hasil ke CSV
                    output_name = os.path.splitext(filename)[0] + "_final.csv"
                    output_path = os.path.join(output_folder, output_name)
                    df.to_csv(output_path, index=False)

                # Hapus file sementara setelah diproses
                os.remove(bil_file_path)

                # Juga hapus file .hdr jika ada
                hdr_path = bil_file_path.replace(".bil", ".hdr")
                if os.path.exists(hdr_path):
                    os.remove(hdr_path)

    except Exception as e:
        print(f"❌ Gagal memproses {tar_file}: {e}")

print("🎉 Semua file selesai diproses.")


In [ ]:
excel_file = '/content/drive/MyDrive/Shared/Koordinat CH.xlsx'
df_points = pd.read_excel(excel_file)
pos = df_points["Nama Pos"].values
lat_points = df_points["Y"].values
lon_points = df_points["X"].values
point_coords = np.array([lon_points, lat_points]).T

def process_csv_file(file_path):
    df = pd.read_csv(file_path)

    # Ambil lon, lat, ch
    lons = np.sort(df["lon"].unique())
    lats = np.sort(df["lat"].unique())

    # Bentuk grid dari data (diasumsikan grid lengkap)
    grid_data = df.pivot_table(index="lat", columns="lon", values="ch").values
    # Hati-hati: pastikan urutan lat dari besar ke kecil sesuai orientasi data
    if lats[0] > lats[-1]:
        lats = lats[::-1]
        grid_data = grid_data[::-1, :]

    interpolator = RegularGridInterpolator((lats, lons), grid_data, bounds_error=False, fill_value=np.nan)

    interpolated = interpolator(point_coords[:, [1, 0]])  # karena (lat, lon)

    # Fallback ke nearest neighbor kalau ada NaN
    if np.isnan(interpolated).any():
        valid_mask = ~np.isnan(df["ch"])
        valid_points = df.loc[valid_mask, ["lon", "lat"]].values
        valid_values = df.loc[valid_mask, "ch"].values

        nan_mask = np.isnan(interpolated)
        nearest_idx = np.argmin(cdist(point_coords[nan_mask], valid_points), axis=1)
        interpolated[nan_mask] = valid_values[nearest_idx]

    # Ambil tanggal dari nama file
    filename = os.path.basename(file_path)
    match = re.search(r'(\d{6})_final\.csv$', filename)
    if match:
        date = datetime.strptime(match.group(1), "%Y%m")
    else:
        date = None

    return pd.DataFrame({
        'pos': pos,
        'lon': lon_points,
        'lat': lat_points,
        'precipitation': interpolated,
        'date': date
    })

# Proses semua file CSV
folder = "/content/final_output/"
all_csv_files = sorted(glob(os.path.join(folder, '*_final.csv'), recursive=True))

results = []

for file_path in all_csv_files:
    df_interp = process_csv_file(file_path)

    # Simpan per file (opsional)
    output_path = file_path.replace("_final.csv", "_interpolated.csv")
    df_interp.to_csv(output_path, index=False)

    results.append(df_interp)

print("✅ Semua file selesai diproses.")

# Gabung semua (opsional)
df_all = pd.concat(results, ignore_index=True)
df_all.to_csv('/content/drive/MyDrive /01-CURAH_HUJAN/Bulanan/CHIRPS 56.csv', index=False)


✅ Semua file selesai diproses.


## IMERG

In [ ]:
file_path = '/content/drive/MyDrive/GPM_3IMERGM_1998/3B-MO.MS.MRG.3IMERG.19980101-S000000-E235959.01.V07B.HDF5'

excel_file = '/content/drive/MyDrive/Shared/Koordinat CH.xlsx'
df_points = pd.read_excel(excel_file)
pos = df_points["Nama Pos"].values
lat_points = df_points["Y"].values  # Latitude
lon_points = df_points["X"].values  # Longitude
point_coords = np.array([lon_points, lat_points]).T  # bentuk (N, 2)


def process_imerg_file(file_path):
    with h5py.File(file_path, 'r') as f:
        precip = f["Grid"]["precipitation"][0, :, :]  # (lon, lat)
        lat = f["Grid"]["lat"][:]
        lon = f["Grid"]["lon"][:]

    # Buat interpolator: RegularGridInterpolator expects (x, y) = (lon, lat)
    interpolator = RegularGridInterpolator((lon, lat), precip, bounds_error=False, fill_value=np.nan)

    # Interpolasi bilinear
    interpolated = interpolator(point_coords)

    # Nearest neighbor untuk NaN
    if np.isnan(interpolated).any():
        valid_mask = ~np.isnan(precip)
        lon_grid, lat_grid = np.meshgrid(lon, lat, indexing='ij')
        valid_points = np.array([lon_grid[valid_mask], lat_grid[valid_mask]]).T
        valid_values = precip[valid_mask]

        nan_mask = np.isnan(interpolated)
        nearest_idx = np.argmin(cdist(point_coords[nan_mask], valid_points), axis=1)
        interpolated[nan_mask] = valid_values[nearest_idx]

    # Ambil tanggal dari nama file
    filename = os.path.basename(file_path)
    try:
        date_str = filename.split('.')[4].split('-')[0]  # Contoh: '19980101'
        date = datetime.strptime(date_str, "%Y%m%d")
    except Exception:
        date = None

    return pd.DataFrame({
        'pos': pos,
        'lon': lon_points,
        'lat': lat_points,
        'precipitation': interpolated,
        'date': date
    })

folder = "/content/drive/MyDrive/03-IMERG_GPM/"
all_files = sorted(glob(os.path.join(folder,'**', "*.HDF5")))

results = []

for file_path in all_files:
    df_interp = process_imerg_file(file_path)

    # Simpan per file (opsional)
    output_path = file_path.replace(".HDF5", "_interpolated.csv")
    df_interp.to_csv(output_path, index=False)

    # Atau gabungkan semua ke satu list
    results.append(df_interp)
print("Selesai")

# (Opsional) Gabung semua hasil
df_all = pd.concat(results, ignore_index=True)
df_all.to_csv("/content/drive/MyDrive/Bulanan/interpolated_all.csv", index=False)


## GSMAP

In [ ]:
base_dir = '/content/drive/MyDrive /01-CURAH_HUJAN/04-GSMAP/'
output_dir = '/content/drive/MyDrive/Shared/00 Hujan/04-GSMaP_csv'  # folder tujuan hasil

os.makedirs(output_dir, exist_ok=True)

# Cari semua file .dat.gz
file_list = glob(os.path.join(base_dir, '**', '*.dat.gz'), recursive=True)

for file_path in file_list:
    # Ekstrak tanggal dari nama file
    basename = os.path.basename(file_path)
    yyyymm = basename.split('.')[1]
    date = pd.to_datetime(yyyymm, format="%Y%m")

    # Baca file binary
    with gzip.open(file_path, "rb") as f:
        data = np.frombuffer(f.read(), dtype='<f4')

    rain_rate = data[:4320000].copy()
    valid_pixel_count = data[4320000:]

    # Handle missing value
    rain_rate[rain_rate == -999.9] = np.nan

    # Buat grid
    lat = np.linspace(59.95, -59.95, 1200)
    lon = np.linspace(-179.95, 179.95, 3600)
    lon_grid, lat_grid = np.meshgrid(lon, lat)

    # DataFrame
    df = pd.DataFrame({
        "lon": lon_grid.flatten(),
        "lat": lat_grid.flatten(),
        "rain_rate": rain_rate,
        "valid_pixel_count": valid_pixel_count
    })
    df["total_rain_month"] = df["rain_rate"] * df["valid_pixel_count"]
    df["date"] = date

    # 🎯 Filter hanya Indonesia
    df = df[(df["lat"] >= -11) & (df["lat"] <= 6) &
            (df["lon"] >= 95) & (df["lon"] <= 141)]

    # Simpan per file
    output_file = os.path.join(output_dir, f"gsmap_{yyyymm}.csv")
    df.to_csv(output_file, index=False)
print("Selesai")


Selesai


In [ ]:
# 1. Load titik-titik target
excel_file = '/content/drive/MyDrive/Shared/Koordinat CH.xlsx'
df_points = pd.read_excel(excel_file)
pos = df_points["Nama Pos"].values
lat_points = df_points["Y"].values
lon_points = df_points["X"].values
points = np.array([lat_points, lon_points]).T

# 2. Siapkan folder CSV GSMaP
csv_dir = '/content/drive/MyDrive/Shared/00 Hujan/04-GSMaP_csv'
csv_files = sorted(glob(os.path.join(csv_dir, "*.csv")))

# 3. Simpan hasil akhir
interpolated_all = []

# 4. Loop semua file GSMaP per bulan
for file in csv_files:
    df = pd.read_csv(file)

    # Ambil hanya Indonesia (opsional, karena sudah dipotong)
    lat = np.sort(df['lat'].unique())[::-1]  # descending
    lon = np.sort(df['lon'].unique())        # ascending

    # Buat grid 2D untuk total_rain_month
    grid_values = df.pivot(index='lat', columns='lon', values='total_rain_month').values

    # Interpolasi Bilinear
    interpolator = RegularGridInterpolator((lat, lon), grid_values, bounds_error=False, fill_value=np.nan)
    interp_values = interpolator(points)

    # Cek NaN, ganti dengan nearest neighbor
    nan_mask = np.isnan(interp_values)
    if nan_mask.any():
        valid_mask = ~np.isnan(grid_values)
        lon_grid, lat_grid = np.meshgrid(lon, lat)
        valid_points = np.array([lat_grid[valid_mask], lon_grid[valid_mask]]).T
        valid_values = grid_values[valid_mask]

        if len(valid_points) > 0:
            nearest_idx = np.argmin(cdist(points[nan_mask], valid_points), axis=1)
            interp_values[nan_mask] = valid_values[nearest_idx]

    # Simpan hasil bulan ini
    yyyymm = os.path.basename(file).split("_")[1].split(".")[0]
    df_month = pd.DataFrame({
        "date": pd.to_datetime(yyyymm, format="%Y%m"),
        'pos': pos,
        "lon": lon_points,
        "lat": lat_points,
        "rain": interp_values
    })
    interpolated_all.append(df_month)

# 5. Gabung semua bulan
df_final = pd.concat(interpolated_all, ignore_index=True)

# 6. Simpan ke Excel / CSV
df_final.to_csv('/content/drive/MyDrive/Interpolated_GSMaP_Indonesia.csv', index=False)
print("Interpolasi selesai dan disimpan ✅")


Interpolasi selesai dan disimpan ✅


## ERA5

In [ ]:
df_points = pd.read_excel('/content/drive/MyDrive/Shared/Koordinat CH.xlsx')

ds = xr.open_mfdataset('/content/drive/MyDrive/Shared/00 Hujan/ERA5/*.nc', combine="by_coords")

# === 1. Baca koordinat titik dari Excel ===
lat_points = df_points["Y"].values
lon_points = df_points["X"].values
point_coords = np.array([lat_points, lon_points]).T

# === 2. Baca data NetCDF ===
tp = ds["tp"]  # (valid_time, lat, lon)
lats = ds["latitude"].values
lons = ds["longitude"].values
times = pd.to_datetime(ds["valid_time"].values)

# Tambahkan koordinat waktu datetime
tp = tp.assign_coords(valid_time=times)

# Tambahkan koordinat tahun dan bulan untuk groupby
tp.coords["year"] = ("valid_time", tp["valid_time"].dt.year.data)
tp.coords["month"] = ("valid_time", tp["valid_time"].dt.month.data)

# === 3. Agregasi bulanan di grid ===
tp_monthly = tp.groupby(["year", "month"]).sum(dim="valid_time")  # hasil: (year, month, lat, lon)

# === 4. Interpolasi ke titik Excel per bulan ===
results = []

for i in range(tp_monthly["year"].size):
    for j in range(tp_monthly["month"].size):
        try:
            tp_sel = tp_monthly.isel(year=i, month=j)
            year_val = int(tp_sel["year"].values)
            month_val = int(tp_sel["month"].values)
            tp_vals = tp_sel.values  # (lat, lon)

            # Interpolator bilinear
            interpolator = RegularGridInterpolator((lats, lons), tp_vals, bounds_error=False, fill_value=np.nan)
            interpolated = interpolator(point_coords)

            # Nearest Neighbor fallback
            nan_mask = np.isnan(interpolated)
            if nan_mask.any():
                valid_mask = ~np.isnan(tp_vals)
                lat_grid, lon_grid = np.meshgrid(lats, lons, indexing='ij')
                valid_points = np.array([lat_grid[valid_mask], lon_grid[valid_mask]]).T
                valid_values = tp_vals[valid_mask]
                nearest_idx = np.argmin(cdist(point_coords[nan_mask], valid_points), axis=1)
                interpolated[nan_mask] = valid_values[nearest_idx]

            # Simpan ke DataFrame
            df_out = df_points.copy()
            df_out["tp_interpolated"] = interpolated * 1000 # dari m ke mm
            df_out["tahun"] = year_val
            df_out["bulan"] = month_val
            results.append(df_out)

        except Exception as e:
            print(f"Skip year={i}, month={j} karena error: {e}")
            continue

# Gabung semua
df_all = pd.concat(results, ignore_index=True)


In [ ]:
df_all.to_csv('/content/drive/MyDrive /01-CURAH_HUJAN/Bulanan/Era5.csv', index = False)

## GRID

In [ ]:
# 🔹 Konfigurasi folder
source_folder = '/content/drive/MyDrive /01-CURAH_HUJAN/02-CHIRPS/Unduhan'
extract_folder = '/content/ekstrak'
output_folder = '/content/drive/MyDrive /01-CURAH_HUJAN/Grid/02-CHIRPS'
os.makedirs(extract_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# 🔹 Batas koordinat wilayah Indonesia
lat_min, lat_max = -12.0, -5.5
lon_min, lon_max = 105.0, 115.0

tar_files = [f for f in os.listdir(source_folder) if f.endswith(".tar.gz")]

def extract_date_flexible(filename):
    # Modifikasi sesuai pola nama file kamu
    import re
    match = re.search(r"\d{4}[\._-]?\d{2}", filename)
    return match.group(0).replace("_", "-").replace(".", "-") if match else "unknown"

for tar_file in tar_files:
    try:
        tar_path = os.path.join(source_folder, tar_file)
        with tarfile.open(tar_path, "r:gz") as tar:
            members = tar.getmembers()
            for m in members:
              # Ekstrak semua file .bil dan file pendukung lainnya
              relevant_exts = [".bil", ".hdr", ".blw", ".prj"]
              extracted_names = []

            for member in members:
                if any(member.name.endswith(ext) for ext in relevant_exts):
                    member.name = os.path.basename(member.name)  # Hilangkan path folder
                    tar.extract(member, path=extract_folder)
                    extracted_names.append(member.name)


            # Proses semua file .bil yang telah diekstrak
            for filename in extracted_names:
                if not filename.endswith(".bil"):
                    continue

                bil_file_path = os.path.join(extract_folder, filename)

                with rasterio.open(bil_file_path) as dataset:
                    data = dataset.read(1)
                    data[data == dataset.nodata] = -10

                    transform = dataset.transform
                    nrows, ncols = dataset.height, dataset.width

                    lons = np.array([transform[2] + i * transform[0] for i in range(ncols)])
                    lats = np.array([transform[5] + j * transform[4] for j in range(nrows)])

                    lon_grid, lat_grid = np.meshgrid(lons, lats)

                    df = pd.DataFrame({
                        "lon": lon_grid.flatten(),
                        "lat": lat_grid.flatten(),
                        "ch": data.flatten()
                    })

                    df["date"] = extract_date_flexible(filename)
                    df["date"] = pd.to_datetime(df["date"], format="%Y%m", errors="coerce")

                    # Simpan hasil ke CSV
                    output_name = os.path.splitext(filename)[0] + "_grid.csv"
                    output_path = os.path.join(output_folder, output_name)
                    df.to_csv(output_path, index=False)

                # Hapus file sementara setelah diproses
                os.remove(bil_file_path)

                # Juga hapus file .hdr jika ada
                hdr_path = bil_file_path.replace(".bil", ".hdr")
                if os.path.exists(hdr_path):
                    os.remove(hdr_path)

    except Exception as e:
        print(f"❌ Gagal memproses {tar_file}: {e}")

print("🎉 Semua file selesai diproses.")


KeyboardInterrupt: 